In [1]:
from sklearn.cluster import KMeans #k-means
import numpy as np
import pandas as pd
import cv2
import os
from PIL import Image 
import scipy.misc
import imageio
import glob
import csv

In [2]:
def create_words_counter_per_sentence(trainFileName):
    train_data = pd.read_csv(trainFileName,delimiter=";")
    sentences_train = [sentence[:len(sentence) -1 ] for sentence in train_data.orth]
    sentences = sentences_train
    numberofWords = []
    for sentence in sentences:
        sentence = sentence.split()
        counter = 0
        for  word in sentence:   
            word = word.lower()
            if((word != "[silence]" and word != "[silence")):
#                 print (word,counter)
                counter+=1
        numberofWords.append(counter)
    del train_data
    del sentences
    del sentences_train
    return numberofWords

In [3]:
# para determinar la duración de los videos
def getLength(filename):
    v=cv2.VideoCapture(filename)
    v.set(cv2.CAP_PROP_POS_AVI_RATIO,1)
    segs = v.get(cv2.CAP_PROP_POS_MSEC)
    segs=segs/1000
    return segs
def duraciones_de_videos(path):
    duraciones=[]    
    for folder in os.listdir(str(path)):
        for file in os.listdir(path+folder):
            video=path+folder+"/"+file
            lg=getLength(video)
            duraciones.append(lg)
    return duraciones

In [4]:
def getMean_noZeros(lista):
    suma=0
    contador=0
    for i in lista:
        if(i!=0):
            contador=contador+1
            suma=suma+i
#             print("I = "+str(i)) 
    return (suma/contador)

In [ ]:
# Dense Optical Flow in OpenCV y creo el vector de vectores
filename = "../../corpus/train.sentences.pronunciations.multi.translations.csv"
totalWords = create_words_counter_per_sentence(filename) 
videos="Files/videos_nuevos/"
duraciones = duraciones_de_videos(videos)

path='Files/videos_nuevos/'
array_vecs=[]
array_imgs=[]
array_movs=[]
# vector_de_frames=[]
# vector_de_frames=np.zeros(1)
vector_de_angulos=[]
vector_de_angulos_por_video=[]
posicion=0
contador=0
for folder in os.listdir(path):    
    for file in os.listdir(str(path+folder)):
        print(file)
        cap = cv2.VideoCapture(str(path+folder+'/'+file))
        ret, frame1 = cap.read()
        prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
        hsv = np.zeros_like(frame1)
        hsv[...,1] = 255

        while(1):
            ret, frame2 = cap.read()
            try:
                next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
                flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
                
                mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])

                array_vecs.append(([ang]))

                hsv[...,0] = ang*180/np.pi/2
                vector_de_angulos=np.array(hsv[...,0]).flatten()
                angulo_promedio_por_frame=getMean_noZeros(vector_de_angulos)
                vector_de_angulos_por_video.append(angulo_promedio_por_frame)
                contador=contador+1
                print(contador)

                prvs = next
            except:
                break
        tamaño_vec=len(vector_de_angulos_por_video)
        with open('vectores_frames_x_video.csv', 'w', newline='') as csvfile:
            filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
            for i in range(totalWords[posicion]):
                comienzo=int((tamaño_vec/totalWords[posicion])*(i))
                fin=int((tamaño_vec/totalWords[posicion])*(i+1))
                filewriter.writerow(vector_de_angulos_por_video[comienzo:fin])
#     vector_de_frames=[]
    posicion=posicion+1    
#     if(posicion==1):
#         break
cap.release()
cv2.destroyAllWindows()

001.avi
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


In [ ]:
# with open('putos_todos.csv', 'r') as f:
#     reader = csv.reader(f, delimiter=',', quotechar='|')
# #     reader = csv.reader(f)
 
#     # read file row by row
#     i=0
#     for row in reader:
#         i=i+len(row)
#         print(len(row))
#     print(i)
# # print(vector_de_ángulos_por_frame)
   